# Proyecto: El Desafío de Don Rene

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

### Cuerpo Docente:

- Profesores: Matías Rojas - Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

*Por favor, lean detalladamente las instrucciones de la tarea antes de empezar a escribir.*


----

## Reglas

- Fecha de entrega: 13/12/2022 (Sin atrasos)
- **Grupos de 2 personas.**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Estrictamente prohibida la copia. 
- Pueden usar cualquier material del curso que estimen conveniente.


---

## El desafio de Don Rene


Don Rene, afamado *influencer* y próximo emprendedor de IA, decide que será una buena idea darle un giro a su vida y dedicarse al rubro del asesoramiento de inversionistas en la industria de los videojuegos. 

El futuro empresario plantea que el éxito relativo de una inversión en una desarrolladora de juegos debe ser analizado desde un punto de vista basado en evidencia y no en la intuición. 
Por dicha razón, plantea que para el sería ideal que existiera una máquina que, dada las características de un nuevo videojuego (su género, el estudio que lo desarrolla, su historia, etc...), indique si este será potencialmente una inversión rentable o no.

Rene está convencido que el éxito de una inversión en videojuegos estar relacionada por dos características muy relevantes de estos una vez que salen al mercado:

**1. La potencial evaluación (Positivo, Negativo, etc...) que le den los usuarios que jugarán el juego.**

**2. Las potenciales ventas del juego.**

Si bien la idea puede sonar excelente, Don Rene carece en su totalidad de una formación en Ciencia de los Datos, por lo que decide ir en ayuda de expertos para implementar su idea.

Sin embargo, como fiel discipulo del libre mercado, decide no contratar a un equipo en particular, si no que tener la libertad de elegir entre muchos equipos que compiten entre si para saber cuál contratar. Para esto recurre a una triquiñuela recurrentemente utilizada en data science: Establecer una competencia abierta y contratar a los que cumplan mejor sus requisitos.


Para esto, Don Rene abre una competencia en la plataforma Codalab (plataforma similar a Kaggle) la cuál, espera que se replete de buenas propuestas. Los equipos que mejor evaluaciones obtengan (los primeros 5 de cada tabla) serán contratados y retribuidos con un cupón canjeable con la friolera cantidad de 1 punto bonus para el proyecto en el curso MDS7202.





### Definición Formal del Problema

El objetivo de este proyecto es relativamente sencillo, pero en extremo ambicioso: Tendrán que aplicar todo lo aprendido hasta este momento con el fin de solucionar 2 problemas distintos: 

1. **Clasificación de potenciales evaluaciones con las que los jugadores evaluarán los juegos**. Las posibles clases que deben asignar a cada juego son `('Negative', 'Mixed', 'Mostly Positive', 'Positive', 'Very Positive')`. La métrica de evaluación utilizada para medir la clasificación es `f1_weighted`.
2. **Regresión de las potenciales ventas que tendrán los juegos**. La métrica de evaluación utilizada para medir la clasificación es `r_2`.

Para esto, ustedes contarán un dataset que cuenta con diversa información de los juegos de la plataforma Steam (tales como fecha de lanzamiento, logros disponibles, plataforma, desarrollador, distribuidor, tags, etc...) más las etiquetas y valores a predecir.


El objetivo final es que participen en la competencia generada en Codalab. Una competencia de Data Science funciona generalmente de la siguiente manera: 

1. Se plantea un problema que los equipos deben resolver.
2. Se provee de datos de entrenamiento a los equipos para que generen modelos que resuelvan el problema.
3. Se provee de datos de prueba que los equipos deberán predecir con los modelos creados. Una vez predichos, los equipos deben subir los archivos a la plataforma, la cuál los evaluará y publicará en un tablero disponible para todos los participantes.

Por este motivo, se les provee de dos datasets: `train.pickle` y `test.pickle`.

- Por una parte, `train.pickle` está preparado que lo utilizen como conjunto de entrenamiento del modelo; por lo que incluye las etiquetas y valores por predecir. 

- Por otra parte, la evaluación de la competencia será hecha usando `test.pickle`. Es decir, este dataset solo contiene características de los juegos y ustedes deberán predecir tanto las potenciales evaluaciones como las posibles ventas de estos y subir sus resultados.

- Además, tendrán sample subissions, es decir, un ejemplo de como subir las evaluaciones.

Obviamente para subir archivos a la competencia deberán registrarse en Codalab. **Puden participar máximo 5 veces en la competencia** (la idea es evitar que se sobreajusten a los datos que deben predecir)

**IMPORTANTE**: Para la clasificación no usen la cantidad de ventas (target de la regresión) como atributo. Por otro lado, para la regresión no utilicen las evaluaciones como atributo para predecir. Infringir estas reglas implicará en no contar el puntaje de la competencia como tampoco poder participar de los bonus por los tops.


---

## Secciones Requeridas en el Informe

La siguiente lista detalla las secciones que debe contener su notebook para resolver el proyecto. Es importante que al momento de desarrollar cada una de las secciones, estas sean escritas en un formato tipo **informe**, donde describan detalladamente cada uno de los puntos realizados.

### 1. Introducción [0.5 Puntos]

*Esta sección es literalmente una muy breve introducción con todo lo necesario para entender que hicieron en su proyecto.*

- Describir brevemente ambos problemas planteados (clasificación y regresión).
- Describir brevemente los datos de entrada que les provee el problema.
- Describir qué métricas se ocupan para la evaluación del problema y por qué se utilizan.
- [Escribir al final] Describir brevemente el modelo que usaron para resolver el problema (incluyendo las transformaciones intermedias de datos).
- [Escribir al final] Indicar si lograron resolver el problema a través de su modelo. Indiquen además si creen que los resultados de su mejor modelo son aceptables y como les fue con respecto al resto de los equipos.

### 2. Análisis Exploratorio de Datos [1.5 Punto]


*La idea de esta sección es que exploren el dataset y obtengan una idea de como son los datos de su problema para que en la siguientes secciones, puedan modelarlos.*

- Analizar cantidad de datos nulos, tipos de datos, duplicados, distribuciones de las variables a través de histogramas.
- Generar visualizaciones de las interacciones (como por ejemplo, una scatter matrix) en las distintas variables.
- Ver las correlaciones entre las distintas variables y los valores faltantes de cada una de estas. 
- Proyectar los datos con PCA, T-SNE o UMAP para ver si existen relaciones entre las distintas variables de forma bi o tri dimensional.
- Reportar los patrones y relaciones interesantes.
- No restringirse a lo mencionado anteriormente.

### 3. Preparación de Datos [1 Punto]

*Esta sección consiste en generar los distintos pasos para preparar sus datos con el fin de luego poder crear su modelo.*

- Descartar datos nulos y outliers.
- Preprocesar datos categóricos y ordinales.
- Escalar/estandarizar datos numéricos.
- Codificar texto.
- Setear las transformaciones en un `ColumnTransformer`.
- Probar las transformaciones.

### 4. Creación de los *Baseline* [1.0 Punto]

*En esta sección crearán el modelo más básico posible que resuelva el problema. La idea de este modelo usarlo como comparación para que en el siguiente paso lo puedan mejorar.*

- Generar un pipeline para la clasificación un clasificador relativamente sencillo (a su elección).
- Generar un pipeline para la regresión con  un regresor relativamente sencillo (a su elección).
- Evaluar ambos modelos según las métricas de evaluación y reportar.

### 5. Optimización de los Modelos [1.5 Puntos]

*Aquí deberán mejorar los modelos al variar los hiperparámetros que están ocupando. No se les pide 100% de nada, si se va a demorar la edad del universo en explorar su grilla, entonces achíquenla a algo que sepan que va a terminar.*

- Usar **`HalvingGridSearchCV`** para tunear hipermarámetros.
- Agregar técnicas de reduccion de dimensionalidad como `PCA` o de seleccion de atributos, como también usar mejores clasificadores y explorar sus hiperparámetros. 
- Probar distintos parámetros para las transformaciones de datos, seleccion de atributos, clasificadores, etc...
- Probar modelos basados en gradient boosting/bagging.  
- Probar activando/descativando los procesadores de texto, de categorías, etc...
- Recuerden setear la búsqueda para optimizar la métrica que se evalua en la competencia.

- **Generen grillas decentes que terminen ojalá antes que cierre el curso.**
    
### 6. Concluir [0.5 Punto] 


*Aquí deben escribir una breve conclusión del trabajo que hicieron en donde incluyan (pero no se limiten) a responder las siguientes preguntas:*

- ¿Pudieron resolver exitosamente el problema?
- ¿Son aceptables los resultados obtenidos?
- ¿Como fue el rendimiento del baseline?
- ¿Pudieron optimizar el baseline?
- ¿Que tanto mejoro el baseline con respecto a sus optimizaciones?
- ¿Estuvieron conformes con sus resultados?
- ¿Creen que hayan mejores formas de modelar el problema?
- ¿Es didáctico usar una competencia?
- ¿En general, qué aprendieron del proyecto?¿Qué no aprendieron y les gustaría haber aprendido?
- Etc...


### 7. Bonificación por superar el baseline de la competencia. [por determinar]

*Este punto es solo informativo, no deben escribir nada aquí*

---

## Esquema de la Tarea


Pueden usar el siguiente esquema para organizar la tarea (y borrar todo lo anterior).
Obviamente **no deben limitarse a lo que está escrito en esta**: puede incrementar en caso de más técnicas y obviar algunas partes en caso que alguna y otro punto no aplique a su problema.

Pueden borrar las instrucciones anteriores y quedarse solo con lo que viene a continuación.


---
---
<br>

<br>

<br>



# Proyecto

### Equipo:

- Juan Carlos Araya
- Javiera Palominos

- \<Nombre de usuarios en Codalab\>

- \<Nombre del Equipo en Codalab\>

### Link de repositorio de GitHub: `\<http://....\>`




## 1. Introducción

El objetivo de este proyecto consiste en ...

Los datos que proveen es un dataset con X ejemplos que describen una observación de ... 
Son N atributos y la variables objetivos son de tipo ... y ...

La primera tarea se evalua en base a la métrica ... ya que esta permite medir ...

Nuestra propuesta para resolver el problema consistieron en modelo basado en...

Nuestro modelo cumplió/no cumplió las expectativas...

---
## 2. Análisis Exploratorio de Datos

In [342]:
# Libreria Core
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [343]:
df_train = pd.read_pickle("train.pickle")
df_train = df_train.astype(
    {
        "english": bool,
        "short_description": str,
    }
)
display(df_train.head())

dims = df_train.shape

print("El DataFrame tiene", dims[0], "filas y", dims[1] ,"columnas")
print("Columnas Dataset: ", list(df_train.columns), "\n")
print("Tipos de Variables: ", "\n", df_train.dtypes, "\n")
print("Cantidad de valores nulos por columna:")
print(df_train.isna().sum())
print("Presenta un total de", len(df_train[df_train.duplicated(subset=["name"])]) ,"filas con nombres duplicados.")
print("Descripción numérica del Dataframe:")
display(df_train.describe(include=[np.number])) #cambiar object a number


#print("Descripción de las columnas: ")

# df_train = pd.read_csv(...)

# px.histograms(...)

#cov = df_train.cov()

#sns.heatmap(cov)

#ax = sns.heatmap(abs(df_train.corr()), annot=True)
# px.scatter(proyecciones)
# etc...

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description,estimated_sells,rating
0,An Aspie Life,2018-03-29,True,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,0,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,3914,Mixed
1,GhostControl Inc.,2014-06-06,True,bumblebee,Application Systems Heidelberg,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Casual;Indie;Simulation;Strategy,Turn-Based;Indie;Simulation,53,65,10.99,Manage a team of ghosthunters and free London ...,10728,Mixed
2,Deponia,2012-08-06,True,Daedalic Entertainment,Daedalic Entertainment,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Point & Click;Comedy,19,217,6.99,"In Deponia, the world has degenerated into a v...",635792,Positive
3,Atlas Reactor,2016-10-04,True,Trion Worlds,Trion Worlds,windows,0,Multi-player;Online Multi-Player;Steam Achieve...,Free to Play;Strategy,Free to Play;Multiplayer;Strategy,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,253864,Positive
4,CHUCHEL,2018-03-07,True,Amanita Design,Amanita Design,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie,Adventure;Indie;Casual,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,49818,Mostly Positive


El DataFrame tiene 7881 filas y 16 columnas
Columnas Dataset:  ['name', 'release_date', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'tags', 'achievements', 'average_playtime', 'price', 'short_description', 'estimated_sells', 'rating'] 

Tipos de Variables:  
 name                   object
release_date           object
english                  bool
developer              object
publisher              object
platforms              object
required_age            int64
categories             object
genres                 object
tags                   object
achievements            int64
average_playtime        int64
price                 float64
short_description      object
estimated_sells         int64
rating               category
dtype: object 

Cantidad de valores nulos por columna:
name                 0
release_date         0
english              0
developer            0
publisher            0
platforms            0
required_age         

,required_age,achievements,average_playtime,price,estimated_sells
count,7881.00000,7881.000000,7881.000000,7881.000000,7.881000e+03
mean,0.78924,43.170156,439.296790,8.431342,2.105767e+05
std,3.55538,265.399206,3303.162083,8.755668,1.513926e+06
min,0.00000,0.000000,0.000000,0.000000,3.600000e+03
25%,0.00000,0.000000,0.000000,1.990000,9.724000e+03
50%,0.00000,15.000000,27.000000,6.990000,2.150800e+04
75%,0.00000,35.000000,251.000000,11.390000,7.357300e+04
max,18.00000,9821.000000,190625.000000,78.990000,7.944129e+07


In [344]:
df_train[df_train["average_playtime"]==0]

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description,estimated_sells,rating
0,An Aspie Life,2018-03-29,True,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,0,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,3914,Mixed
8,INFERNIUM,2018-04-03,True,Carlos Coronado,Carlos Coronado,windows,0,Single-player;Steam Achievements;Full controll...,Adventure;Indie,Adventure;Indie;Horror,24,0,19.49,"Infernium, a survival horror approach to Pac-M...",3914,Mixed
13,NEKOPALIVE,2016-05-31,True,NEKO WORKs,Sekai Project,windows,16,Single-player;Partial Controller Support,Casual;Free to Play,Anime;Free to Play;VR,0,0,0.00,The catgirls of La Soleil takes to the stage a...,62764,Very Positive
16,Puzzler World,2010-05-28,True,Ideas Pad,Puzzler,windows,0,Single-player;Steam Achievements;Steam Leaderb...,Casual,Casual;Puzzle;Singleplayer,14,0,4.99,Puzzler World brings over 1200 puzzles of 8 po...,26544,Positive
17,The Bluecoats: North vs South,2015-02-02,True,Little World Studio,Microids,windows;mac,0,Single-player;Multi-player;Shared/Split Screen...,Action;Strategy,Strategy;Action;Casual,0,0,3.99,1861. The Civil War has been declared in the U...,16182,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7868,Creepy Road,2018-05-18,True,Groovy Milk,Groovy Milk;GrabTheGames,windows;mac,0,Single-player;Steam Achievements;Full controll...,Action;Adventure;Indie,Action;Indie;Singleplayer,18,0,9.99,The world has gone mad and Flint Trucker must ...,4142,Mixed
7871,Riptide GP: Renegade,2016-07-26,True,Vector Unit,Vector Unit,windows,0,Single-player;Multi-player;Shared/Split Screen...,Racing,Racing;Multiplayer;Action,24,0,6.99,Experience a future of illicit hydrojet racing...,8996,Positive
7872,Bladed Fury,2018-12-17,True,NEXT Studios,NEXT Studios,windows,0,Single-player;Steam Achievements;Full controll...,Action;Adventure;Casual;Indie;RPG,Action;Female Protagonist;Indie,24,0,7.19,"An action game set in the Ancient China, bring...",55214,Positive
7878,Moe Jigsaw,2018-03-23,True,ARES Inc.,ARES Inc.,windows,0,Single-player;Steam Achievements;Steam Trading...,Casual;Indie,Casual;Nudity;Indie,72,0,2.89,&quot;Moe Jigsaw&quot; is the definitive versi...,10906,Mostly Positive




Al efectuar el análisis exploratorio de datos pudimos notar lo siguiente:

El atributo A tenía X datos nulos y Y duplicados... que podrían potencialmente dificultar el análisis.

La variable A se distribuye de forma normal. En cambio la B en forma exponencial. La variable C es categórica.

Los atributos A y B interactúan de P forma, mientras que los C y D de Q forma...

Los atributos A y C se correlacionan con las variables de salida Y de forma creciente... B no tiene correlación con la salida.

Además, encontramos que al proyectar en 2D ...

...

---

## 3. Preparación de Datos

Para preparar nuestros datos, primero eliminamos duplicados y outliers...:

In [345]:
## Código
x = df_train.iloc[:,:14]
y = df_train.iloc[:,14:16]

In [346]:
y.iloc[:,1:2]

,rating
0,Mixed
1,Mixed
2,Positive
3,Positive
4,Mostly Positive
...,...
7876,Positive
7877,Very Positive
7878,Mostly Positive
7879,Mostly Positive


Después, imputamos ciertos datos

In [347]:
## Código
def SuperDummier(df_in, col):
    df = df_in.copy()
    df[col] = df[col].apply(lambda x: x.split(";"))
    s = pd.Series(df[col])
    l = pd.get_dummies(s.apply(pd.Series).stack(), prefix=col).max(level=0)
    df = pd.concat((df.drop(col,1),l),axis=1)
    return df

X = SuperDummier(x,"publisher")
X = SuperDummier(X,"developer")
X = SuperDummier(x,"platforms")
X = SuperDummier(X,"categories")
X = SuperDummier(X,"genres")
X = SuperDummier(X,"tags")

In [348]:
X

,name,release_date,english,developer,publisher,required_age,achievements,average_playtime,price,short_description,...,tags_Wargame,tags_Warhammer 40K,tags_Web Publishing,tags_Werewolves,tags_Western,tags_Word Game,tags_World War I,tags_World War II,tags_Wrestling,tags_Zombies
0,An Aspie Life,2018-03-29,True,Bradley Hennessey;Joe Watson,EnderLost Studios,0,23,0,0.00,One day your roommate Leaves for no reason. Yo...,...,0,0,0,0,0,0,0,0,0,0
1,GhostControl Inc.,2014-06-06,True,bumblebee,Application Systems Heidelberg,0,53,65,10.99,Manage a team of ghosthunters and free London ...,...,0,0,0,0,0,0,0,0,0,0
2,Deponia,2012-08-06,True,Daedalic Entertainment,Daedalic Entertainment,0,19,217,6.99,"In Deponia, the world has degenerated into a v...",...,0,0,0,0,0,0,0,0,0,0
3,Atlas Reactor,2016-10-04,True,Trion Worlds,Trion Worlds,0,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,...,0,0,0,0,0,0,0,0,0,0
4,CHUCHEL,2018-03-07,True,Amanita Design,Amanita Design,0,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7876,KOEWOTAYORINI / 声之寄托,2018-03-26,False,あみそ組,Pujia8 Studio;Gamera Game,0,20,65,0.00,“喂喂，求求你啦！救救我！” ——你接到这样一个电话 发出这样的求救，给你打电话的人，是一名...,...,0,0,0,0,0,0,0,0,0,0
7877,Montaro,2016-07-25,True,JCKSLAP,MBDL,0,15,174,0.79,Montaro is a DOGE.,...,0,0,0,0,0,0,0,0,0,0
7878,Moe Jigsaw,2018-03-23,True,ARES Inc.,ARES Inc.,0,72,0,2.89,&quot;Moe Jigsaw&quot; is the definitive versi...,...,0,0,0,0,0,0,0,0,0,0
7879,Drunkn Bar Fight,2016-11-28,True,The Munky,The Munky,0,0,0,10.99,"VR PARTY GAMEDrunkn Bar Fight is a simple, imm...",...,0,0,0,0,0,0,0,0,0,0


Luego, preparamos un `ColumnTransformer` que prepara los datos:

In [349]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kuanki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kuanki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [350]:
## Código
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

preprocessing = ColumnTransformer(
    transformers=[
        #('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['name']),
        ('StandardScaler', StandardScaler(),['required_age','price']), 
        ('MinMaxScaler',  MinMaxScaler(),['achievements','average_playtime'])
    ]
)

---

## 4. Baseline

Adjuntamos las transformaciones anteriores en un solo `Pipeline` y agregamos al final un clasificador sencillo `ClasificadorX` 


In [351]:
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectPercentile, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

dummy = Pipeline(
    steps=[
        ("Preprocessing", preprocessing),
        ("RFC", RandomForestClassifier())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(x, y.iloc[:,1:2], test_size=0.05)

dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

          Mixed       0.26      0.27      0.26        83
Mostly Positive       0.20      0.15      0.17        80
       Negative       0.26      0.23      0.24        62
       Positive       0.36      0.42      0.39       116
  Very Positive       0.20      0.22      0.21        54

       accuracy                           0.28       395
      macro avg       0.26      0.26      0.25       395
   weighted avg       0.27      0.28      0.27       395



In [352]:
dffff = pd.DataFrame(y_test)
dffff["rating_pred"] = y_pred
dffff

,rating,rating_pred
4329,Very Positive,Mixed
4261,Mixed,Mixed
287,Mostly Positive,Negative
6101,Mostly Positive,Positive
2136,Mostly Positive,Mixed
...,...,...
7491,Positive,Mixed
6509,Positive,Mostly Positive
6983,Positive,Positive
6753,Negative,Very Positive


In [353]:
## Código

# pipeline_clf = [('etapa1', etapa1), ..., ('clf', Clasificador)]
# pipeline_rg = [('etapa1', etapa1), ..., ('clf', Regresor)]

# Evalauación
# classfication_report(...)
# ....

---

## 5. Optimización del Modelo

Ahora, agregamos selección y/o reducción de atributos y probaremos una combinación de diferentes parámetros para mejorar nuestro modelo:


*Nota*: Procuren hacer Gridsearch sobre espacios de búsqueda razonables. Es decir, no estén 3 días buscando la mejor configuración...

In [354]:
# grilla_parametros = [{'seleccion': SelectPercentile , 'percentile': [25, 50, 75], 'clf': clf_1}, 
#                      {'clf': clf_2, 'steps'; n},
#                      {...}, 
#                     ]
# gs_2 = HalvingGridSearchCV(pipeline_clf, grilla_parametros, metric='la_metrica_que_optimiza_el_problema')

Pudimos ver que mejor modelo mejora/no mejora los resultados obtenidos en el baseline.

In [355]:
# gs_2.best_...

---

## 6. Conclusiones

Algunas respuestas que podrían plantearse pueden ser:

- ¿Pudieron resolver exitosamente el problema?
- ¿Son aceptables los resultados obtenidos?


- ¿Como fue el rendimiento del baseline?
- ¿Pudieron optimizar el baseline?
- ¿Que tanto mejoro el baseline con respecto a sus optimizaciones?


- ¿Estuvieron conformes con sus resultados?
- ¿Creen que hayan mejores formas de modelar el problema?
- ¿Creen que fue buena idea usar una competencia de por medio?
- ¿En general, qué aprendieron en el pryecto?¿Qué no aprendieron y les gustaría haber aprendido?
- Etc...

**OJO** si usted decide responder parte de estas preguntas, debe redactarlas en un formato de informe y no responderlas directamente.

---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [356]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")